In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus']=False

# sql 함수 사용
from pyspark.sql.functions import when, udf, col, regexp_replace
# sql 타입변경
from pyspark.sql.types import DoubleType,IntegerType, StringType
import pyspark.sql.functions as F

In [2]:
sc = SparkContext('local') # 'spark://ip:7070'
sqlCtx = SQLContext( sc )

In [9]:
df = sqlCtx.read.csv(r'c:\data\crime_in_Seoul.csv',header=True,encoding='euc-kr',inferSchema=True)


In [17]:
# rdd.collect() Row형태로
df.select('관서명').rdd.collect()

[Row(관서명='중부서'),
 Row(관서명='종로서'),
 Row(관서명='남대문서'),
 Row(관서명='서대문서'),
 Row(관서명='혜화서'),
 Row(관서명='용산서'),
 Row(관서명='성북서'),
 Row(관서명='동대문서'),
 Row(관서명='마포서'),
 Row(관서명='영등포서'),
 Row(관서명='성동서'),
 Row(관서명='동작서'),
 Row(관서명='광진서'),
 Row(관서명='서부서'),
 Row(관서명='강북서'),
 Row(관서명='금천서'),
 Row(관서명='중랑서'),
 Row(관서명='강남서'),
 Row(관서명='관악서'),
 Row(관서명='강서서'),
 Row(관서명='강동서'),
 Row(관서명='종암서'),
 Row(관서명='구로서'),
 Row(관서명='서초서'),
 Row(관서명='양천서'),
 Row(관서명='송파서'),
 Row(관서명='노원서'),
 Row(관서명='방배서'),
 Row(관서명='은평서'),
 Row(관서명='도봉서'),
 Row(관서명='수서서')]

In [16]:
df.select('관서명').rdd.map(lambda v:v['관서명']).collect()

['중부서',
 '종로서',
 '남대문서',
 '서대문서',
 '혜화서',
 '용산서',
 '성북서',
 '동대문서',
 '마포서',
 '영등포서',
 '성동서',
 '동작서',
 '광진서',
 '서부서',
 '강북서',
 '금천서',
 '중랑서',
 '강남서',
 '관악서',
 '강서서',
 '강동서',
 '종암서',
 '구로서',
 '서초서',
 '양천서',
 '송파서',
 '노원서',
 '방배서',
 '은평서',
 '도봉서',
 '수서서']

### 컬럼 객체 멤버함수
- .cast()
- .isin()
- .like()
- .rlike()

In [18]:
# where (문자열 or 컬럼객체)
df.where(df['관서명'].isin('중부서','종로서')).show()

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [19]:
df.where(df['관서명'].like('%북%')).show()

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|성북서|        2|        2|        2|        1|       86|       71|      953|      409|    1,194|    1,015|
|강북서|        7|        8|       14|       13|      153|      126|    1,434|      618|    2,649|    2,348|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [28]:
# rlike 정규식 패턴 사용
# df.where(df['관서명'].rlike('^강')).show() # 강으로 시작되는 데이터
# df.where(df['관서명'].rlike('^마|동')).show() # 마로 시작되거나 동이 들어간 데이터
# df.where(df['관서명'].rlike('[북남]')).show() # '북'이 있거나 '남'이 있거나
df.where(df['관서명'].rlike('강[북남]')).show() # '강북' 또는 '강남'

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|강북서|        7|        8|       14|       13|      153|      126|    1,434|      618|    2,649|    2,348|
|강남서|        3|        3|       15|       12|      300|      225|    2,411|      984|    2,465|    2,146|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+

